-sandbox
<img src="https://files.training.databricks.com/images/Apache-Spark-Logo_TM_200px.png" style="float: left: margin: 20px"/>

# Working with Time Windows

## In this lesson you:
* Use sliding windows to aggregate over chunks of data rather than all data
* Apply watermarking to throw away stale old data that you do not have space to keep
* Plot live graphs using `display`

## Audience
* Primary Audience: Data Engineers
* Secondary Audience: Data Scientists, Software Engineers

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Getting Started</h2>

Run the following cell to configure our "classroom."

In [0]:
%run "./Includes/Classroom-Setup"

Initialized classroom variables & functions...

Datasets are already mounted to /mnt/training from wasbs://training@dbtraineastus.blob.core.windows.net/

Created user-specific database

Using the database jaeykos_hotmail_com_db .

All done!

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Streaming Aggregations</h2>

Continuous applications often require near real-time decisions on real-time, aggregated statistics.

Some examples include
* Aggregating errors in data from IoT devices by type
* Detecting anomalous behavior in a server's log file by aggregating by country.
* Doing behavior analysis on instant messages via hash tags.

However, in the case of streams, you generally don't want to run aggregations over the entire dataset.

### What problems might you encounter if you aggregate over a stream's entire dataset?

While streams have a definitive start, there conceptually is no end to the flow of data.

Because there is no "end" to a stream, the size of the dataset grows in perpetuity.

This means that your cluster will eventually run out of resources.

Instead of aggregating over the entire dataset, you can aggregate over data grouped by windows of time (say, every 5 minutes or every hour).

This is referred to as windowing

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Windowing</h2>

If we were using a static DataFrame to produce an aggregate count, we could use `groupBy()` and `count()`.

Instead we accumulate counts within a sliding window, answering questions like "How many records are we getting every second?"

**Sliding windows** 

The windows overlap and a single event may be aggregated into multiple windows. 

**Tumbling Windows**

The windows do not overlap and a single event will be aggregated into only one window. 

The diagram below shows sliding windows. 

The following illustration, from the <a href="https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html" target="_blank">Structured Streaming Programming Guide</a> guide, helps us understanding how it works:

<img src="http://spark.apache.org/docs/latest/img/structured-streaming-window.png">

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Event Time vs Receipt Time</h2>

**Event Time** is the time at which the event occurred in the real world.

**Event Time** is **NOT** something maintained by the Structured Streaming framework.

At best, Structured Streaming only knows about **Receipt Time** - the time a piece of data arrived in Spark.

### What are some examples of **Event Time**? **of Receipt Time**?

#### Examples of *Event Time*:
* The timestamp recorded in each record of a log file
* The instant at which an IoT device took a measurement
* The moment a REST API received a request

#### Examples of *Receipt Time*:
* A timestamp added to a DataFrame the moment it was processed by Spark
* The timestamp extracted from an hourly log file's file name
* The time at which an IoT hub received a report of a device's measurement
  - Presumably offset by some delay from when the measurement was taken

### What are some of the inherent problems with using **Receipt Time**?

The main problem with using **Receipt Time** is going to be with accuracy. For example:

* The time between when an IoT device takes a measurement vs when it is reported can be off by several minutes.
  - This could have significant ramifications to security and health devices, for example
* The timestamp embedded in an hourly log file can be off by up to one hour making correlations to other events extremely difficult
* The timestamp added by Spark as part of a DataFrame transformation can be off by hours to weeks to months depending on when the event occurred and when the job ran

### When might it be OK to use **Receipt Time** instead of **Event Time**?

When accuracy is not a significant concern - that is **Receipt Time** is close enough to **Event Time**

One example would be for IoT events that can be delayed by minutes but the resolution of your query is by days or months (close enough)

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Windowed Streaming Example</h2>

For this example, we will examine the files in `/mnt/training/sensor-data/accelerometer/time-series-stream.json/`.

Each line in the file contains a JSON record with two fields: `time` and `action`

New files are being written to this directory continuously (aka streaming).

Theoretically, there is no end to this process.

Let's start by looking at the head of one such file:

In [0]:
%fs head dbfs:/mnt/training/sensor-data/accelerometer/time-series-stream.json/file-0.json

[Truncated to first 65536 bytes]
{"time":1469501107,"action":"Open"}
{"time":1469501147,"action":"Open"}
{"time":1469501202,"action":"Open"}
{"time":1469501219,"action":"Open"}
{"time":1469501225,"action":"Open"}
{"time":1469501234,"action":"Open"}
{"time":1469501245,"action":"Open"}
{"time":1469501246,"action":"Open"}
{"time":1469501248,"action":"Open"}
{"time":1469501256,"action":"Open"}
{"time":1469501264,"action":"Open"}
{"time":1469501266,"action":"Open"}
{"time":1469501267,"action":"Open"}
{"time":1469501269,"action":"Open"}
{"time":1469501271,"action":"Open"}
{"time":1469501282,"action":"Open"}
{"time":1469501285,"action":"Open"}
{"time":1469501291,"action":"Open"}
{"time":1469501297,"action":"Open"}
{"time":1469501303,"action":"Open"}
{"time":1469501322,"action":"Open"}
{"time":1469501335,"action":"Open"}
{"time":1469501344,"action":"Open"}
{"time":1469501346,"action":"Open"}
{"time":1469501349,"action":"Open"}
{"time":1469501357,"action":"Open"}
{"time":1469501366,"action":"Open"}
{"time":1469501371,"action":"Open"}
{"time":1469501375,"action":"Open"}
{"time":1469501375,"action":"Open"}
{"time":1469501381,"action":"Open"}
{"time":1469501392,"action":"Open"}
{"time":1469501402,"action":"Open"}
{"time":1469501407,"action":"Open"}
{"time":1469501410,"action":"Open"}
{"time":1469501420,"action":"Open"}
{"time":1469501424,"action":"Open"}
{"time":1469501438,"action":"Open"}
{"time":1469501442,"action":"Close"}
{"time":1469501462,"action":"Open"}
{"time":1469501480,"action":"Open"}
{"time":1469501488,"action":"Open"}
{"time":1469501489,"action":"Open"}
{"time":1469501491,"action":"Open"}
{"time":1469501503,"action":"Open"}
{"time":1469501505,"action":"Open"}
{"time":1469501509,"action":"Open"}
{"time":1469501513,"action":"Open"}
{"time":1469501517,"action":"Open"}
{"time":1469501520,"action":"Open"}
{"time":1469501525,"action":"Open"}
{"time":1469501533,"action":"Open"}
{"time":1469501539,"action":"Open"}
{"time":1469501540,"action":"Open"}
{"time":1469501541,"action":"Open"}
{"time":1469501543,"action":"Open"}
{"time":1469501544,"action":"Open"}
{"time":1469501545,"action":"Close"}
{"time":1469501545,"action":"Open"}
{"time":1469501547,"action":"Open"}
{"time":1469501552,"action":"Open"}
{"time":1469501557,"action":"Open"}
{"time":1469501559,"action":"Open"}
{"time":1469501560,"action":"Open"}
{"time":1469501560,"action":"Open"}
{"time":1469501565,"action":"Open"}
{"time":1469501566,"action":"Open"}
{"time":1469501574,"action":"Open"}
{"time":1469501575,"action":"Open"}
{"time":1469501575,"action":"Open"}
{"time":1469501578,"action":"Open"}
{"time":1469501581,"action":"Open"}
{"time":1469501584,"action":"Open"}
{"time":1469501600,"action":"Open"}
{"time":1469501601,"action":"Open"}
{"time":1469501603,"action":"Open"}
{"time":1469501610,"action":"Open"}
{"time":1469501620,"action":"Open"}
{"time":1469501621,"action":"Open"}
{"time":1469501625,"action":"Open"}
{"time":1469501625,"action":"Close"}
{"time":1469501626,"action":"Open"}
{"time":1469501631,"action":"Open"}
{"time":1469501632,"action":"Open"}
{"time":1469501632,"action":"Open"}
{"time":1469501638,"action":"Open"}
{"time":1469501643,"action":"Open"}
{"time":1469501646,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501662,"action":"Open"}
{"time":1469501663,"action":"Open"}
{"time":1469501667,"action":"Open"}
{"time":1469501674,"action":"Open"}
{"time":1469501675,"action":"Open"}
{"time":1469501678,"action":"Close"}
{"time":1469501680,"action":"Open"}
{"time":1469501685,"action":"Open"}
{"time":1469501686,"action":"Open"}
{"time":1469501689,"action":"Open"}
{"time":1469501691,"action":"Open"}
{"time":1469501694,"action":"Open"}
{"time":1469501696,"action":"Close"}
{"time":1469501702,"action":"Open"}
{"time":1469501703,"action":"Open"}
{"time":1469501704,"action":"Open"}
{"time":1469501706,"action":"Open"}
{"time":1469501706,"action":"Open"}
{"time":1469501710,"action":"Open"}
{"time":1469501715,"action":"Open"}
{"

-sandbox

### Define the Schema for the streaming content

Let's try to analyze these files interactively.

First configure a schema.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> The schema must be specified for file-based Structured Streams.
Because of the simplicity of the schema, we can use the simpler, DDL-formatted, string representation of the schema.

In [0]:
inputPath = "dbfs:/mnt/training/sensor-data/accelerometer/time-series-stream.json/"

jsonSchema = "time timestamp, action string"

### Define a streaming Dataframe

With the schema defined, we can create the initial DataFrame `inputDf` and then `countsDF` which represents our aggregation:

In [0]:
from pyspark.sql.functions import window, col

inputDF = (spark
  .readStream                                 # Returns an instance of DataStreamReader
  .schema(jsonSchema)                         # Set the schema of the JSON data
  .option("maxFilesPerTrigger", 1)            # Treat a sequence of files as a stream, one file at a time
  .json(inputPath)                            # Specifies the format, path and returns a DataFrame
)

countsDF = (inputDF
  .groupBy(col("action"),                     # Aggregate by action...
           window(col("time"), "1 hour"))     # ...then by a 1 hour window
  .count()                                    # For the aggregate, produce a count
  .select(col("window.start").alias("start"), # Elevate field to column
          col("action"),                      # Include count
          col("count"))                       # Include action
  .orderBy(col("start"), col("action"))       # Sort by the start time and action
)

### View Results

To view the results of our query, pass the DataFrame `countsDF` to the `display()` function.

In [0]:
display(countsDF)

start,action,count
2016-07-26T02:00:00.000+0000,Close,11
2016-07-26T02:00:00.000+0000,Open,179
2016-07-26T03:00:00.000+0000,Close,344
2016-07-26T03:00:00.000+0000,Open,1001
2016-07-26T04:00:00.000+0000,Close,815
2016-07-26T04:00:00.000+0000,Open,999
2016-07-26T05:00:00.000+0000,Close,323
2016-07-26T05:00:00.000+0000,Open,328
2016-07-26T13:00:00.000+0000,Close,699
2016-07-26T13:00:00.000+0000,Open,656


### Performance Considerations

If you run that query, as is, it will take a surprisingly long time to start generating data. What's the cause of the delay?

If you expand the **Spark Jobs** component, you'll see something like this:

<img src="https://files.training.databricks.com/images/structured-streaming-shuffle-partitions-200.png"/>

It's our `groupBy()`. `groupBy()` causes a _shuffle_, and, by default, Spark SQL shuffles to 200 partitions. In addition, we're doing a _stateful_ aggregation: one that requires Structured Streaming to maintain and aggregate data over time.

When doing a stateful aggregation, Structured Streaming must maintain an in-memory _state map_ for each window within each partition. For fault tolerance reasons, the state map has to be saved after a partition is processed, and it needs to be saved somewhere fault-tolerant. To meet those requirements, the Streaming API saves the maps to a distributed store. On some clusters, that will be HDFS. Databricks uses the DBFS.

That means that every time it finishes processing a window, the Streaming API writes its internal map to disk. The write has some overhead, typically between 1 and 2 seconds.

### What's the cause of the delay?
* `groupBy()` causes a **shuffle**
* By default, this produces **200 partitions**
* Plus a **stateful aggregation** to be maintained **over time**

This results in :
* Maintenance of an **in-memory state map** for **each window** within **each partition**
* Writing of the state map to a fault-tolerant store
  * On some clusters, that will be HDFS
  * Databricks uses the DBFS
* Around 1 to 2 seconds overhead

### Shuffle Partition Best Practices

One way to reduce this overhead is to reduce the number of partitions Spark shuffles to.

In most cases, you want a 1-to-1 mapping of partitions to cores for streaming applications.

### Run query with proper setting for shuffle partitions

Rerun the query below and notice the performance improvement.

Once the data is loaded, render a line graph with
* **Keys** is set to `start`
* **Series groupings** is set to `action`
* **Values** is set to `count`

In [0]:
spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism)

display(countsDF)

start,action,count
2016-07-26T02:00:00.000+0000,Close,11
2016-07-26T02:00:00.000+0000,Open,179
2016-07-26T03:00:00.000+0000,Close,344
2016-07-26T03:00:00.000+0000,Open,1001
2016-07-26T04:00:00.000+0000,Close,815
2016-07-26T04:00:00.000+0000,Open,999
2016-07-26T05:00:00.000+0000,Close,1003
2016-07-26T05:00:00.000+0000,Open,1000
2016-07-26T06:00:00.000+0000,Close,1011
2016-07-26T06:00:00.000+0000,Open,993


### Stop all Streams

When you are done, stop all the streaming jobs.

In [0]:
for s in spark.streams.active: # Iterate over all active streams
  s.stop()                     # Stop the stream

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2565828149947187> in <module> 
 1 for s in spark . streams . active : # Iterate over all active streams 
 ----> 2 s . stop ( ) # Stop the stream 

 /databricks/spark/python/pyspark/sql/streaming.py in stop (self) 
 155 """Stop this streaming query.
 156 """
 --> 157 self . _jsq . stop ( ) 
 158 
 159 def explain ( self , extended = False ) : 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 115 def deco ( * a , ** kw ) : 
 116 try : 
 --> 117 return f ( * a , ** kw ) 
 118 except py4j . protocol . Py4JJavaError as e : 
 119 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o611.stop.
: java.util.concurrent.TimeoutException: Stream Execution thread for stream display_query_5 [id = e8a8ec5a-3b77-4051-9cd0-608f34656bac, runId = deb16bd5-3671-4dad-8821-5cecffeb354e] failed to stop within 15000 milliseconds (specified by spark.sql.streaming.stopTimeout). See the cause on what was being executed in the streaming query thread.
	at org.apache.spark.sql.execution.streaming.StreamExecution.interruptAndAwaitExecutionThreadTermination(StreamExecution.scala:520)
	at org.apache.spark.sql.execution.streaming.MicroBatchExecution.stop(MicroBatchExecution.scala:212)
	at org.apache.spark.sql.execution.streaming.StreamingQueryWrapper.stop(StreamingQueryWrapper.scala:67)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: org.apache.spark.SparkException: The stream thread was last executing:
	at java.net.SocketInputStream.socketRead0(Native Method)
	at java.net.SocketInputStream.socketRead(SocketInputStream.java:116)
	at java.net.SocketInputStream.read(SocketInputStream.java:171)
	at java.net.SocketInputStream.read(SocketInputStream.java:141)
	at sun.security.ssl.SSLSocketInputRecord.read(SSLSocketInputRecord.java:476)
	at sun.security.ssl.SSLSocketInputRecord.readHeader(SSLSocketInputRecord.java:470)
	at sun.security.ssl.SSLSocketInputRecord.bytesInCompletePacket(SSLSocketInputRecord.java:70)
	at sun.security.ssl.SSLSocketImpl.readApplicationRecord(SSLSocketImpl.java:1364)
	at sun.security.ssl.SSLSocketImpl.access$300(SSLSocketImpl.java:73)
	at sun.security.ssl.SSLSocketImpl$AppInputStream.read(SSLSocketImpl.java:973)
	at java.io.BufferedInputStream.fill(BufferedInputStream.java:246)
	at java.io.BufferedInputStream.read1(BufferedInputStream.java:286)
	at java.io.BufferedInputStream.read(BufferedInputStream.java:345)
	at sun.net.www.http.HttpClient.parseHTTPHeader(HttpClient.java:743)
	at sun.net.www.http.HttpClient.parseHTTP(HttpClient.java:678)

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Problem with Generating Many Windows</h2>

We are generating a window for every 1 hour aggregate.

_Every window_ has to be separately persisted and maintained.

Over time, this aggregated data will build up in the driver.

The end result being a massive slowdown if not an OOM Error.

### How do we fix that problem?

One simple solution is to increase the size of our window (say, to 2 hours).

That way, we're generating fewer windows.

But if the job runs for a long time, we're still building up an unbounded set of windows.

Eventually, we could hit resource limits.

<h2><img src="https://files.training.databricks.com/images/105/logo_spark_tiny.png"> Watermarking</h2>

A better solution to the problem is to define a cut-off.

A point after which Structured Streaming will commit windowed data to sink, or throw it away if the sink is console or memory as `display()` mimics.

That's what _watermarking_ allows us to do.

### Refining our previous example

Below is our previous example with watermarking.

We're telling Structured Streaming to keep no more than 2 hours of aggregated data.

In [0]:
watermarkedDF = (inputDF
  .withWatermark("time", "2 hours")           # Specify a 2-hour watermark
  .groupBy(col("action"),                     # Aggregate by action...
           window(col("time"), "1 hour"))     # ...then by a 1 hour window
  .count()                                    # For each aggregate, produce a count
  .select(col("window.start").alias("start"), # Elevate field to column
          col("action"),                      # Include count
          col("count"))                       # Include action
  .orderBy(col("start"), col("action"))       # Sort by the start time
)
display(watermarkedDF)                        # Start the stream and display it

start,action,count
2016-07-26T02:00:00.000+0000,Close,11
2016-07-26T02:00:00.000+0000,Open,179
2016-07-26T03:00:00.000+0000,Close,344
2016-07-26T03:00:00.000+0000,Open,1001
2016-07-26T04:00:00.000+0000,Close,815
2016-07-26T04:00:00.000+0000,Open,999
2016-07-26T05:00:00.000+0000,Close,323
2016-07-26T05:00:00.000+0000,Open,328
2016-07-26T13:00:00.000+0000,Close,699
2016-07-26T13:00:00.000+0000,Open,656


### Example Details

In the example above,
* Data received 2 hour _past_ the watermark will be dropped.
* Data received within 2 hours of the watermark will never be dropped.

More specifically, any data less than 2 hours behind the latest data processed till then is guaranteed to be aggregated.

However, the guarantee is strict only in one direction.

Data delayed by more than 2 hours is not guaranteed to be dropped; it may or may not get aggregated.

The more delayed the data is, the less likely the engine is going to process it.

### Stop all the streams

In [0]:
for s in spark.streams.active: # Iterate over all active streams
  s.stop()                     # Stop the stream


## Next steps

Start the next lesson, [Structured Streaming with Azure EventHubs]($./3.Streaming-With-Event-Hubs-Demo)